In [3]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI() # model="gpt-3.5-turbo-instruct"을 기본으로 사용, 그외 temperature, max_token 등 튜닝이 가능.

b = chat.predict("How many planets are there?")

b

'There are 8 planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.'

In [4]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage # 인간, AI, LLM 설정을 제공하기 위한 메시지

messages = [
    SystemMessage(
        content="Your are a geography expert. And you only reply in Spanish.",
        ), 
    AIMessage(
        content="Hola, me llamo Laura!"
        ),
    HumanMessage(
        content="What is the distance between Mexico and Thailand. Also, what is your name?"
        )
]

chat.predict_messages(messages)

# 이전에 string을 predict 했다면 이번에는 message의 리스트를 predict하게 함. 
# 일종의 설정을 넘겨주어 AI가 답변하게 만들었음.


AIMessage(content='La distancia entre México y Tailandia es de aproximadamente 16,000 kilómetros. Mi nombre es Laura. ¿Hay algo más en lo que te pueda ayudar?')

In [10]:
# langchain은 prompt에 많은 utility를 제공하고 있음. 
# prompt끼리 결합, 저장 또는 불러올 수 있음. 변수 설정 도중에 실수는 하지 않았는지 검증도 할 수 있음. 
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate # template을 string으로 받냐, message로 받냐의 차이

chat = ChatOpenAI(temperature=0.1)

template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}?"
)

# ChatPromptTemplate를 통해 template을 system, ai, human 이렇게 나누어서 커스텀하기 편해짐
template = ChatPromptTemplate.from_messages([
    ("system", "Your are a geography expert. And you only reply in {language}."), 
    ("ai", "Hola, me llamo {name}!"),
    ('human', "What is the distance between {country_a} and {country_b}. Also, what is your name?")
])

# 변수들을 검증할 수 있음. 
prompt = template.format_messages(language="Greek", 
                         name='Socrates', 
                         country_a="Mexico", 
                         country_b='Thailand')

chat.predict_messages(prompt)


AIMessage(content='Γεια σας! Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα. Το όνομά μου είναι Σωκράτης. Πώς μπορώ να βοηθήσω;')

In [13]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))
p = CommaOutputParser()
p.parse("Hello, how, are, you")


['Hello', 'how', 'are', 'you']

In [17]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lower case. Do NOT reply with anything else."),
    ("human", "{question}")]
)

prompt = template.format_messages(max_items=10, question="What are the colors?")

result = chat.predict_messages(prompt)

p = CommaOutputParser()

p.parse(result.content)

['red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'pink',
 'black',
 'white',
 'brown']

In [19]:
chain = template | chat | CommaOutputParser()
# 여러 chain을 결합해서도 사용할 수 있음. ex. all = chain_1 | chain_2

chain.invoke({"max_items": 5, "question":"What are the pokemons?"})

['pikachu', 'charmander', 'bulbasaur', 'squirtle', 'jigglypuff']

In [25]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1,
                  streaming=True,
                  callbacks=[
                      StreamingStdOutCallbackHandler()
                      ])

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create eay to follow recipies for any type of cusine with easy to find ingredients."),
    ("human", "I want to cook {cuisine} food.")
])

chef_chain = chef_prompt | chat 



In [26]:
veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it.",
        ),
        ("human", "{recipe}"),
    ]
)
# recipe라는 placeholder를 두고 있는 상태.

veg_chain = veg_chef_prompt | chat


final_chain = {"recipe": chef_chain} | veg_chain 
# chef_chain의 출력값은 veg_chef_prompt의 입력값인 recipe가 되어야함.
# 이게 바로 RunnableMap : langchain이 먼저 chef_chain을 먼저 돌리고 그 응답 결과를 다음 요소에게 전달하는 것. 

final_chain.invoke({"cuisine": "korean"})

Korean cuisine is known for its bold flavors and unique ingredients. One popular dish that you can easily make at home is Bibimbap, a delicious and colorful mixed rice bowl. Here's a simple recipe for Bibimbap:

Ingredients:
- 2 cups cooked short-grain rice
- 1 cup assorted vegetables (such as spinach, carrots, zucchini, mushrooms, bean sprouts)
- 1/2 cup protein of your choice (such as beef, chicken, tofu)
- 2 eggs
- 2 tablespoons soy sauce
- 1 tablespoon sesame oil
- 1 tablespoon gochujang (Korean chili paste)
- 1 tablespoon sesame seeds
- Salt and pepper to taste
- Vegetable oil for cooking

Instructions:
1. Prepare the vegetables: Wash and chop the assorted vegetables into bite-sized pieces. Cook each vegetable separately in a pan with a little vegetable oil until tender. Season with salt and pepper to taste.

2. Cook the protein: Season the protein of your choice with salt and pepper. Cook in a pan with a little vegetable oil until fully cooked. Set aside.

3. Fry the eggs: In a s

AIMessageChunk(content="For a vegetarian version of Bibimbap, you can easily substitute the protein with tofu or tempeh. Here's how you can prepare them:\n\n1. **Tofu**: \n   - Use firm or extra-firm tofu for Bibimbap.\n   - Press the tofu to remove excess water. You can do this by wrapping the tofu block in a clean kitchen towel and placing a heavy object on top for about 15-30 minutes.\n   - Cut the tofu into cubes or slices, whichever you prefer.\n   - Season the tofu with salt and pepper, and you can also marinate it in a mixture of soy sauce, sesame oil, and a bit of gochujang for extra flavor.\n   - Cook the tofu in a pan with a little vegetable oil until it's golden brown and slightly crispy on the outside.\n\n2. **Tempeh**:\n   - Tempeh is another great plant-based protein option for Bibimbap.\n   - Cut the tempeh into cubes or slices.\n   - You can steam the tempeh for about 10 minutes before cooking to reduce its bitterness.\n   - Season the tempeh with salt and pepper, and y